#### Data Preprocessing

In [ ]:
import pandas as pd       
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)
test = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
data=[train,test]

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
train

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment w...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy Hines...
2,"""7759_3""",0,The film starts with a manager Nicholas Bell g...
3,"""3630_4""",0,It must be assumed that those who praised this...
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,"""3453_3""",0,It seems like more consideration has gone into...
24996,"""5064_1""",0,I dont believe they made this film Completely ...
24997,"""10905_3""",0,Guy is a loser Cant get girls needs to build u...
24998,"""10194_3""",0,This 30 minute documentary Buñuel made in the ...


In [ ]:
test.head(5)

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [ ]:
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [ ]:
import re
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [ ]:
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
train['review']=train['review'].apply(denoise_text)


In [ ]:
test['review']=test['review'].apply(denoise_text)

In [ ]:
train.head(2)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."


In [ ]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

In [ ]:
import string
string.punctuation
def remove_pun(text):
  no_pun="".join([char for char in text if char not in string.punctuation])
  return no_pun

for i in data:
  i["review"]=i["review"].apply(lambda x: remove_pun(x))

In [ ]:
train.head(2)

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment w...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy Hines...


In [ ]:
test.head(5)

,id,review
0,"""12311_10""",Naturally in a film whos main themes are of mo...
1,"""8348_2""",This movie is a disaster within a disaster fil...
2,"""5828_4""",All in all this is a movie for kids We saw it ...
3,"""7186_2""",Afraid of the Dark left me with the impression...
4,"""12128_7""",A very accurate depiction of small time mob li...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train=train['review'].values

In [ ]:
Y_train=train['sentiment'].values

In [ ]:
X_test=test['review'].values

## Artificial Neural Networks Application

## Vectorizers :Bag of words

#### Count Vectorizers

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(X_train)
x_train=vec.transform(X_train)
x_test=vec.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Dense(16, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 16)                2270160   
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 1)                 17        
                                                                 
Total params: 2,270,449
Trainable params: 2,270,449
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train, Y_train,epochs=40,verbose=True,batch_size=16)

Epoch 1/40
1360/1360 [==============================] - 6s 5ms/step - loss: 3.4023e-10 - accuracy: 1.0000
Epoch 2/40
1360/1360 [==============================] - 6s 5ms/step - loss: 3.0588e-10 - accuracy: 1.0000
Epoch 3/40
1360/1360 [==============================] - 7s 5ms/step - loss: 2.8636e-10 - accuracy: 1.0000
Epoch 4/40
1360/1360 [==============================] - 6s 5ms/step - loss: 2.6744e-10 - accuracy: 1.0000
Epoch 5/40
1360/1360 [==============================] - 6s 5ms/step - loss: 2.4516e-10 - accuracy: 1.0000
Epoch 6/40
1360/1360 [==============================] - 6s 5ms/step - loss: 2.3674e-10 - accuracy: 1.0000
Epoch 7/40
1360/1360 [==============================] - 7s 5ms/step - loss: 2.2133e-10 - accuracy: 1.0000
Epoch 8/40
1360/1360 [==============================] - 6s 5ms/step - loss: 2.1520e-10 - accuracy: 1.0000
Epoch 9/40
1360/1360 [==============================] - 6s 5ms/step - loss: 1.9318e-10 - accuracy: 1.0000
Epoch 10/40
1360/1360 [=======================

In [ ]:
model.evaluate(x_train,Y_train)

In [ ]:
res=model.predict(x_test)

In [ ]:
## Done

### TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vec = TfidfVectorizer()

In [ ]:
vec.fit(X_train)

TfidfVectorizer()

In [ ]:
x_train=vec.transform(X_train)

In [ ]:
X_test=vec.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, Y_train)

LogisticRegression()

In [ ]:
result = lr.predict(X_test)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )


In [ ]:
## Done